In [1]:
%load_ext autoreload
%load_ext dotenv
%dotenv
!huggingface-cli login --token $HUGGING_FACE_TOKEN

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/izlobin/.cache/huggingface/token
Login successful


In [12]:
# %%capture
# %pip install nltk absl-py rouge_score
# %pip install bleu sacrebleu
# %pip install sacremoses
# %pip install scipy
# %pip install sentencepiece
# %pip install optimum auto-gptq
# %pip install scikit-learn
# %pip install einops
# %pip install bitsandbytes
# %pip install huggingface_hub
# %pip install transformers evaluate gradio datasets chardet cchardet librosa ipython sentencepiece plotly phonemizer
# %pip install accelerate
# %pip install pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint

import evaluate
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from huggingface_hub import HfApi
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BartForCausalLM,
    BartModel,
    BartTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5TokenizerFast,
)

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from utils.dataset import get_iterater_samples_simplified, get_iterater_samples_with_instruction
from utils.metric import calculate_scores
from utils.monitoring import calculate_utilization, format_utilization_narrow, print_utilization

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pprint(f"Device: {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
'Device: cuda'


## Loading models

### Loading custom gpt2 / gpt2-large

In [3]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "gpt2-large"
model_repo = f"openai-community"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")


tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
)
tokenizer.add_eos_token = True
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

print(type(tokenizer))


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=0)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=0)
print(type(model))
# print(model.config)
# print(model)

trained_model_name = "gpt2-large-bnb8-coedit"
trained_model_repo = f"iliazlobin"
trained_model_id = f"{trained_model_repo}/{trained_model_name}"
trained_model_checkpoint = f"{trained_model_repo}/{trained_model_name}"
trained_model_path = f"{trained_model_repo}_{trained_model_name}"
print(
    f"trained_model_name: {trained_model_name},"
    f"trained_model_id: {trained_model_id},"
    f"trained_model_path: {trained_model_path}"
)


adapters_path = f"../model-train/model-{trained_model_repo}_{trained_model_name}"
peft_model = PeftModel.from_pretrained(model, adapters_path)

print(type(peft_model))
print(peft_model.config)
print(peft_model)

origin_model = model
model = peft_model
model_name = trained_model_name
model_repo = trained_model_repo
model_id = trained_model_id
model_checkpoint = trained_model_checkpoint
model_path = trained_model_path
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

model_name: gpt2-large,model_id: openai-community/gpt2-large,model_path: openai-community_gpt2-large
<class 'transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
trained_model_name: gpt2-large-bnb8-coedit,trained_model_id: iliazlobin/gpt2-large-bnb8-coedit,trained_model_path: iliazlobin_gpt2-large-bnb8-coedit
<class 'peft.peft_model.PeftModelForCausalLM'>
GPT2Config {
  "_name_or_path": "openai-community/gpt2-large",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1280,
  "n_head": 20,
  "n_inner": null,
  "n_layer": 36,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": tr

/home/izlobin/.local/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Loading BART
* https://huggingface.co/facebook/bart-large

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BartForConditionalGeneration, BitsAndBytesConfig

model_name = "bart-large"
model_repo = f"facebook"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

tokenizer = BartTokenizer.from_pretrained(model_id)
tokenizer.add_eos_token = True
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
print(type(tokenizer))

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=0)
print(type(model))

model_name: bart-large,model_id: facebook/bart-large,model_path: facebook_bart-large
<class 'transformers.models.bart.tokenization_bart.BartTokenizer'>
<class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BartForConditionalGeneration, BitsAndBytesConfig

model_name = "bart-large-coedit"
model_repo = f"iliazlobin"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

tokenizer = BartTokenizer.from_pretrained(model_id)
tokenizer.add_eos_token = True
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
print(type(tokenizer))

# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=0, max_memory={0: "20GIB"})
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=0)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
print(type(model))

model_name: bart-large-coedit,model_id: iliazlobin/bart-large-coedit,model_path: iliazlobin_bart-large-coedit
<class 'transformers.models.bart.tokenization_bart.BartTokenizer'>
<class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>


In [3]:
model_name = "iliazlobin/bart-grammarly"
model_alias = model_name.replace("/", "_")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
# model = BartModel.from_pretrained(model_name)
# model = BartForCausalLM.from_pretrained(model_name, device_map=0)
model = BartForConditionalGeneration.from_pretrained(model_name, device_map=0)

print(type(tokenizer))
print(type(model))
print(model.config)

<class 'transformers.models.bart.tokenization_bart.BartTokenizer'>
<class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>
BartConfig {
  "_name_or_path": "iliazlobin/bart-grammarly",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
 

### Loading google/t5-large
* https://huggingface.co/google-t5/t5-large

In [6]:
model_name = "t5-large"
model_repo = f"google-t5"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

tokenizer = T5TokenizerFast.from_pretrained(model_id)
# tokenizer.add_eos_token = True
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side = "right"
print(type(tokenizer))

# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=0, max_memory={0: "20GIB"})
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map=0)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
print(type(model))

print(type(tokenizer))
print(type(model))
print(model.config)

model_name: t5-large,model_id: google-t5/t5-large,model_path: google-t5_t5-large
<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>
<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>
<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
T5Config {
  "_name_or_path": "google-t5/t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  

### Loading grammarly/coedit

In [4]:
model_name = "coedit-large"
model_repo = f"grammarly"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")


model = T5ForConditionalGeneration.from_pretrained(model_id, device_map=0)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(type(tokenizer))
print(type(model))
print(model.config)

model_name: coedit-large,model_id: grammarly/coedit-large,model_path: grammarly_coedit-large
<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>
<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
T5Config {
  "_name_or_path": "grammarly/coedit-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_ver

### Loading Gemma

In [18]:
model_name = "google/gemma-2b-it"
# model_name = "google/gemma-7b-it"
# model_name = "google/gemma-7b"
model_alias = model_name.replace("/", "_")

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = T5ForConditionalGeneration.from_pretrained(model_name, device_map=0)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     # torch_dtype=torch.float16,
#     # revision="float16",
#     device_map="auto",
# )

print(model.config)

T5Config {
  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32128
}



### Loading Phi-2
* https://huggingface.co/microsoft/phi-2
* https://huggingface.co/TheBloke/phi-2-GPTQ

In [3]:
# model_name = 'microsoft/phi-2'
model_name = 'TheBloke/phi-2-GPTQ'
model_alias = model_name.replace('/', '_')

torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    trust_remote_code=True,
)

# print(f"model.config.eos_token_id: {model.config.eos_token_id}")
# eos_token_id = 50256 # https://huggingface.co/microsoft/phi-2/blob/main/config.json

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

print(f"tokenizer.eos_token: {tokenizer.eos_token}")
tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     # torch_dtype=torch.float16,
#     # revision="float16",
#     device_map="auto",
# )

print(model.config)

/home/izlobin/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer.eos_token: <|endoftext|>
PhiConfig {
  "_name_or_path": "TheBloke/phi-2-GPTQ",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "TheBloke/phi-2-GPTQ--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "TheBloke/phi-2-GPTQ--modeling_phi.PhiForCausalLM"
  },
  "embd_pdrop": 0.0,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "img_processor": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "phi-msft",
  "n_embd": 2560,
  "n_head": 32,
  "n_head_kv": null,
  "n_inner": null,
  "n_layer": 32,
  "n_positions": 2048,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.1,
    "dataset": null,
    "desc_act": true,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,

### Loading mixtral-8x7B
* https://huggingface.co/mistralai/Mixtral-8x7B-v0.1
* https://huggingface.co/TheBloke/Mixtral-8x7B-v0.1-GPTQ

Quantization
* https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996

In [7]:
# model_name = "mistralai/Mixtral-8x7B-v0.1"
model_name = 'TheBloke/Mixtral-8x7B-v0.1-GPTQ'
model_alias = model_name.replace('/', '_')

torch.cuda.empty_cache()

# from transformers import BitsAndBytesConfig
# quantization_config = BitsAndBytesConfig(load_in_4bit=4)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map=0,
#     # torch_dtype=torch.float16,
#     # load_in_4bit=True,
#     # quantization_config=quantization_config,
# )

# print(f"model.config.eos_token_id: {model.config.eos_token_id}")
# eos_token_id = 50256 # https://huggingface.co/microsoft/phi-2/blob/main/config.json

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=0,
    load_in_4bit=True,
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
)

# print(f"tokenizer.eos_token: {tokenizer.eos_token}")
# tokenizer.pad_token = tokenizer.eos_token


print(model.config)

# text = 'Hello my name is'
# inputs = tokenizer(text, return_tensors='pt')
# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


AttributeError: 'BitsAndBytesConfig' object has no attribute 'get_loading_attributes'

### Loading llama-2

In [3]:
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "TheBloke/Llama-2-7B-GPTQ"
# model_name = "TheBloke/Nous-Hermes-Llama-2-7B-GPTQ"
model_alias = model_name.replace("/", "_")

from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained(model_name, padding_side='left')
model = LlamaForCausalLM.from_pretrained(model_name, device_map=0)

tokenizer.pad_token = tokenizer.eos_token

print(type(tokenizer))
print(type(model))
print(model.config)

# from auto_gptq import exllama_set_max_input_length
# model = exllama_set_max_input_length(model, max_input_length=2400)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/izlobin/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its conten

<class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>
<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
LlamaConfig {
  "_name_or_path": "TheBloke/Llama-2-7B-GPTQ",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_length": 4096,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": null,
    "cache_block_outputs": true,
    "damp_percent": 0.01,
    "dataset": null,
    "desc_act": false,
    "exllama_config": {
      "version": 1
    },
    "group_size": 128,
    "max_input_length": null,
    "model_seqlen": null,
    "mo

## Evaluation

### Grammarly/coedit dataset
* https://huggingface.co/datasets/grammarly/coedit

In [7]:
from datasets import DatasetDict, load_dataset

# full_dataset = load_dataset("grammarly/coedit")
# print(full_dataset)

# train_dataset = load_dataset("grammarly/coedit", split="train[:50000]")
# test_dataset = load_dataset("grammarly/coedit", split="train[10000:]")
# # test_dataset = load_dataset("grammarly/coedit", split="validation")

all_dataset = load_dataset("grammarly/coedit", split="train+validation")
# print(all_dataset)

# print()
# print(f"train set {set(all_dataset['task'])}")
# print(f"total len: {len(all_dataset)}")
# print(f"gec len: {len(all_dataset.filter(lambda x: x['task'] == 'gec'))}")
# print(f"simplification len: {len(all_dataset.filter(lambda x: x['task'] == 'simplification'))}")
# print(f"clarity len: {len(all_dataset.filter(lambda x: x['task'] == 'clarity'))}")
# print(f"coherence len: {len(all_dataset.filter(lambda x: x['task'] == 'coherence'))}")
# print(f"paraphrase len: {len(all_dataset.filter(lambda x: x['task'] == 'paraphrase'))}")
# print(f"neutralize len: {len(all_dataset.filter(lambda x: x['task'] == 'neutralize'))}")
# print()

train_ratio = 0.9
test_ratio = 0.1
# train_ratio = 0.001
# test_ratio = 0.0001

gec_dataset = all_dataset.filter(lambda x: x["task"] == "gec")
train_gec_dataset = gec_dataset.select(range(0, int(train_ratio * len(gec_dataset))))
test_gec_dataset = gec_dataset.select(range(int((1 - test_ratio) * len(gec_dataset)), len(gec_dataset)))

simplification_dataset = all_dataset.filter(lambda x: x["task"] == "simplification")
train_simplification_dataset = simplification_dataset.select(range(0, int(train_ratio * len(simplification_dataset))))
test_simplification_dataset = simplification_dataset.select(
    range(int((1 - test_ratio) * len(simplification_dataset)), len(simplification_dataset))
)

clarity_dataset = all_dataset.filter(lambda x: x["task"] == "clarity")
train_clarity_dataset = clarity_dataset.select(range(0, int(train_ratio * len(clarity_dataset))))
test_clarity_dataset = clarity_dataset.select(range(int((1 - test_ratio) * len(clarity_dataset)), len(clarity_dataset)))

coherence_dataset = all_dataset.filter(lambda x: x["task"] == "coherence")
train_coherence_dataset = coherence_dataset.select(range(0, int(train_ratio * len(coherence_dataset))))
test_coherence_dataset = coherence_dataset.select(
    range(int((1 - test_ratio) * len(coherence_dataset)), len(coherence_dataset))
)

paraphrase_dataset = all_dataset.filter(lambda x: x["task"] == "paraphrase")
train_paraphrase_dataset = paraphrase_dataset.select(range(0, int(train_ratio * len(paraphrase_dataset))))
test_paraphrase_dataset = paraphrase_dataset.select(
    range(int((1 - test_ratio) * len(paraphrase_dataset)), len(paraphrase_dataset))
)

neutralize_dataset = all_dataset.filter(lambda x: x["task"] == "neutralize")
neutralize_dataset_split = int(train_ratio * len(neutralize_dataset))
train_neutralize_dataset = neutralize_dataset.select(range(0, int(train_ratio * len(neutralize_dataset))))
test_neutralize_dataset = neutralize_dataset.select(
    range(int((1 - test_ratio) * len(neutralize_dataset)), len(neutralize_dataset))
)

from datasets import concatenate_datasets

train_dataset = concatenate_datasets(
    [
        train_gec_dataset,
        train_simplification_dataset,
        train_clarity_dataset,
        train_coherence_dataset,
        train_paraphrase_dataset,
        train_neutralize_dataset,
    ]
)
train_dataset = train_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"train set {set(train_dataset['task'])}")
print(train_dataset)

test_dataset = concatenate_datasets(
    [
        test_gec_dataset,
        test_simplification_dataset,
        test_clarity_dataset,
        test_coherence_dataset,
        test_paraphrase_dataset,
        test_neutralize_dataset,
    ]
)
test_dataset = test_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"test set {set(test_dataset['task'])}")
print(test_dataset)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
# dataset = dataset.rename_column("task", "label")
# dataset = dataset.map(
#     lambda item: {
#         "input": item["src"],
#         "reference": item["tgt"],
#         "references": [item["tgt"]],
#     },
#     remove_columns=["src", "tgt", "_id"],
# )
print(dataset)
print(dataset["train"][0])
print(dataset["test"][0])

train set {'paraphrase', 'clarity', 'coherence', 'simplification', 'neutralize', 'gec'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 63703
})
test set {'paraphrase', 'clarity', 'coherence', 'simplification', 'neutralize', 'gec'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 7080
})
DatasetDict({
    train: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 63703
    })
    test: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 7080
    })
})
{'task': 'gec', 'input': 'Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert.', 'reference': 'For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean 

In [5]:
# find the longest sequence in the dataset
max_input_length = max(len(tokenizer.encode(item["input"])) for item in dataset["train"])
print(f"max_input_length train: {max_input_length}")
max_input_length = max(len(tokenizer.encode(item["input"])) for item in dataset["test"])
print(f"max_input_length test: {max_input_length}")

max_input_length train: 173
max_input_length test: 241


In [6]:
from datasets import Dataset

train_lists_map = {}

for task in set(train_dataset['task']):
    train_lists_map[task] = []

for item in dataset["train"]:
    train_lists_map[item["task"]].append(item)

train_dataset_map = {}
for task, items in train_lists_map.items():
    train_dataset_map[task] = Dataset.from_list(items)
# print(train_dataset_map)

train_dataset_dict = DatasetDict(train_dataset_map)
# print(train_dataset_dict)

# for task, ds in train_dataset_dict.items():
#     print(f"{task}: {ds}")

for task in set(train_dataset['task']):
    print(f"train/{task}: {len(train_lists_map[task])}")

train/paraphrase: 14307
train/neutralize: 10143
train/coherence: 9554
train/clarity: 1126
train/gec: 18277
train/simplification: 10296


In [7]:
from datasets import Dataset

test_lists_map = {}

for task in set(train_dataset['task']):
    test_lists_map[task] = []

for item in dataset["test"]:
    test_lists_map[item["task"]].append(item)

test_dataset_map = {}
for task, items in test_lists_map.items():
    test_dataset_map[task] = Dataset.from_list(items)
# print(test_dataset_map)

test_dataset_dict = DatasetDict(test_dataset_map)
print(test_dataset_dict)

# for task, ds in test_dataset_dict.items():
#     print(f"{task}: {ds}")

for task in set(train_dataset['task']):
    print(f"test/{task}: {len(test_lists_map[task])}")

DatasetDict({
    paraphrase: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 1590
    })
    neutralize: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 1127
    })
    coherence: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 1062
    })
    clarity: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 126
    })
    gec: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 2031
    })
    simplification: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 1144
    })
})
test/paraphrase: 1590
test/neutralize: 1127
test/coherence: 1062
test/clarity: 126
test/gec: 2031
test/simplification: 1144


### Test inference

In [8]:
max_batch = 2
max_length = 350

for task, batch in test_dataset_dict.items():
    print()
    print(f">> {task}")
    batch_size = len(batch) if len(batch) < max_batch else max_batch
    input_batch = batch.select(range(batch_size))
    print(f"input: {input_batch['input']}")

    input = tokenizer(input_batch["input"], padding=True, return_tensors="pt").to(device)
    outputs = model.generate(input.input_ids, max_length=max_length)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(f"result: {result}")


>> coherence
input: ['Fix coherence: Outside the town, 6 tourists were reported killed. Official documents indicate that at least 255 local residents were killed, with a further 29 never found.', 'Make the text more cohesive: Whereas at some times (and in some places) a Corps of two divisions was sufficient, at other times 5 or 6 divisions were necessary. Under the Hindenburg regime (from summer 1916), new Corps headquarters were created without organic divisions.']
result: ['Outside the town, 6 tourists were reported killed. Official documents indicate that at least 255 local residents were killed, with a further 29 never found.', 'Whereas at some times (and in some places) a Corps of two divisions was sufficient, at other times 5 or 6 divisions were necessary. Under the Hindenburg regime (from summer 1916), new Corps headquarters were created without organic divisions.']

>> clarity
input: ['Use clearer wording: Canals are waterways channels, or artificial waterways, for water conve

In [ ]:
max_batch = 2
max_length = 350

for task, batch in test_dataset_dict.items():
    print()
    print(f">> {task}")
    batch_size = len(batch) if len(batch) < max_batch else max_batch
    input_batch = batch.select(range(batch_size))
    print(f"input: {input_batch['input']}")

    inputs = tokenizer(input_batch['input'], return_tensors="pt", padding=True).to(device)

    model.config.use_cache = False
    outputs = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id, num_return_sequences=1)
    # outputs = model.generate(
    #     **inputs,
    #     do_sample=True,
    #     top_k=10,
    #     num_return_sequences=1,
    #     pad_token_id=tokenizer.eos_token_id,
    #     # return_attention_mask=True,
    #     max_length=256,
    # )

    trimmed_output = outputs[:, inputs.input_ids.shape[1] :]
    result = tokenizer.batch_decode(trimmed_output, skip_special_tokens=True)
    print(f"result: {result}")


>> simplification
input: ['Simplify this text: When given for short periods, linezolid is a relatively safe antibiotic.', "Make this text less complex: O 'Connor turned professional in 1967."]
result: ['When given for short periods, linezolid is a relatively safe antibiotic.', "O'Connor turned professional in 1967."]

>> coherence
input: ["Fix coherence of the sentence: Silver went to Japan's Miho Takagi in a time of 1: 54.55. Bronze went to Marrit Leenstra of the Netherlands in a time of 1.", 'Improve the cohesiveness of the text: Heather Blonsky a math and science competition for middle-school students. Heather Blonsky is a mentor of a finalist in the 2011 Broadcom MASTERS.']
result: ["Silver went to Japan's Miho Takagi in a time of 1: 54.55. Bronze went to Marrit Leenstra of the Netherlands, who won the event in 1: 55.", 'Heather Blonsky is a mentor of a finalist in the 2011 Broadcom MASTERS. a math and science competition for middle-school students.']

>> gec
input: ['Fix grammar:

### Eval prep

In [8]:
%reload_ext autoreload

from utils.monitoring import calculate_utilization, format_utilization_narrow, print_utilization

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total/trainable params: {total_params}/{total_trainable_params}')

utilization = calculate_utilization()
print(utilization)
utilization_str = format_utilization_narrow(utilization)
print(utilization_str)
print(
    f"total/used/cuda/res/ram(Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)


torch.cuda.empty_cache()

# actual_fraction = 0.95
# available_memory = utilization['total_memory'] - utilization['memory_used']
# recommended_fraction = available_memory / utilization['total_memory']
# torch.cuda.set_per_process_memory_fraction(actual_fraction, 0)

# print(
#     f"Total/used/available memory (Gb): {utilization['total_memory']/1024**3:.2f}/"
#     "{utilization['memory_used']/1024**3:.2f}/{available_memory/1024**3:.2f}"
# )
# print(f'Recommended/actual fraction: {recommended_fraction:.2f}/{actual_fraction:.2f}')

# torch.cuda.empty_cache()
# torch.empty(utilization['total_memory'] // 2, dtype=torch.int8, device='cuda')
# print_utilization()

Total/trainable params: 406291456/406291456
{'total_memory': 84987740160, 'memory_used': 2998403072, 'cuda_allocated': 1625367040, 'cuda_reserved': 1639972864, 'ram_usage': 4190502912}
{'total_memory': '79.15', 'memory_used': '2.79', 'cuda_allocated': '1.51', 'cuda_reserved': '1.53', 'ram_usage': '3.90'}
total/used/cuda/res/ram(Gb): 79.15/2.79/1.51/1.53/3.90


### Eval encoder-decoder models

In [9]:
%%time
%load_ext autoreload

# batch_size = 10
batch_size = 100
max_length = 350


def model_process(batch, idx, **kwargs):
    num_samples = len(batch["input"])
    start_time = time.time()

    model = kwargs.get("model")
    tokenizer = kwargs.get("tokenizer")
    total_samples = kwargs.get("total_samples")

    inputs = tokenizer(batch["input"], max_length=max_length, padding=True, return_tensors="pt").to(device)
    # input_ids = tokenizer(batch['task'], return_tensors="pt").input_ids.to(device)
    # input_ids = tokenizer(item['task'], return_tensors="pt").input_ids
    # outputs = model.generate(input_ids, max_length=512)
    outputs = model.generate(inputs.input_ids, max_length=max_length)
    # print(f"outputs: {outputs}")
    processed = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    end_time = time.time()
    elapsed_time = end_time - start_time
    sps = num_samples / elapsed_time
    sps_str = f"{sps:.2f}"

    utilization = calculate_utilization()
    utilization_str = format_utilization_narrow(utilization)
    print(
        f"{idx[0]}-{idx[-1]}/{total_samples} | total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
        f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']} | "
        f"batch/sps: {num_samples}/{sps_str}"
    )

    return {"processed": processed}


utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

processed_samples_map = {}

for task, samples in test_dataset_dict.items():
    total_samples = len(samples)

    print(f"Processing {total_samples} samples for {task}")

    start_time = time.time()

    processed_samples = samples.map(
        model_process,
        fn_kwargs={
            "model": model,
            "tokenizer": tokenizer,
            "total_samples": total_samples,
        },
        num_proc=1,
        batched=True,
        batch_size=batch_size,
        with_indices=True,
    )

    end_time = time.time()
    elapsed_time = end_time - start_time
    sps = total_samples / elapsed_time
    sps_str = f"{sps:.2f}"

    print(f"Finished processing {total_samples} samples for {task}.")

    utilization = calculate_utilization()
    utilization_str = format_utilization_narrow(utilization)
    print(
        f"{total_samples} | total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
        f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']} | "
        f"sps: {sps_str}"
    )

    processed_samples_map[task] = {
        # "task": task,
        # "samples": samples,
        "samples": processed_samples,
        "total_samples": total_samples,
        "elapsed_time": elapsed_time,
        "sps": sps,
        "batch_size": batch_size,
        "max_length": max_length,
        "utilization": utilization,
    }

# processed_samples = samples.map(model_process, num_proc=torch.cuda.device_count())
# processed_samples = samples.map(
#     model_process,
#     fn_kwargs={
#         "model": model,
#         "tokenizer": tokenizer,
#         "total_samples": total_samples,
#     },
#     num_proc=1,
#     batched=True,
#     batch_size=batch_size,
#     with_indices=True,
# )

processed_gec_samples = processed_samples_map["gec"]["samples"]

pprint(processed_gec_samples)
pprint(processed_gec_samples["processed"][:2])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
total/used/cuda/res/ram (Gb): 79.15/2.79/1.51/1.53/3.91
Processing 1590 samples for paraphrase


Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


0-99/1590 | total/used/cuda/res/ram (Gb): 79.15/72.95/1.52/71.60/6.42 | batch/sps: 100/6.12
100-199/1590 | total/used/cuda/res/ram (Gb): 79.15/17.21/1.52/15.86/6.40 | batch/sps: 100/4.27
200-299/1590 | total/used/cuda/res/ram (Gb): 79.15/17.21/1.52/15.86/6.40 | batch/sps: 100/4.90
300-399/1590 | total/used/cuda/res/ram (Gb): 79.15/40.59/1.52/39.24/6.39 | batch/sps: 100/5.52
400-499/1590 | total/used/cuda/res/ram (Gb): 79.15/40.59/1.52/39.24/6.40 | batch/sps: 100/5.08
500-599/1590 | total/used/cuda/res/ram (Gb): 79.15/45.30/1.52/43.95/6.37 | batch/sps: 100/3.39
600-699/1590 | total/used/cuda/res/ram (Gb): 79.15/45.30/1.52/43.95/6.39 | batch/sps: 100/4.27
700-799/1590 | total/used/cuda/res/ram (Gb): 79.15/45.30/1.52/43.95/6.37 | batch/sps: 100/4.66
800-899/1590 | total/used/cuda/res/ram (Gb): 79.15/45.30/1.52/43.95/6.45 | batch/sps: 100/3.40
900-999/1590 | total/used/cuda/res/ram (Gb): 79.15/45.30/1.52/43.95/6.44 | batch/sps: 100/3.57
1000-1099/1590 | total/used/cuda/res/ram (Gb): 79.15/

Map:   0%|          | 0/1127 [00:00<?, ? examples/s]

...

0-99/126 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.87 | batch/sps: 100/1.15
100-125/126 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.86 | batch/sps: 26/1.50
Finished processing 126 samples for clarity.
126 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.86 | sps: 1.18
Processing 2031 samples for gec


Map:   0%|          | 0/2031 [00:00<?, ? examples/s]

0-99/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.87 | batch/sps: 100/5.64
100-199/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.87 | batch/sps: 100/7.33
200-299/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.86 | batch/sps: 100/10.43
300-399/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.85 | batch/sps: 100/6.23
400-499/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.86 | batch/sps: 100/5.15
500-599/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.93 | batch/sps: 100/7.72
600-699/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.84 | batch/sps: 100/3.87
700-799/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.87 | batch/sps: 100/7.13
800-899/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.86 | batch/sps: 100/6.20
900-999/2031 | total/used/cuda/res/ram (Gb): 79.15/36.99/1.52/35.64/6.87 | batch/sps: 100/4.96
1000-1099/2031 | total/used/cuda/res/ram (Gb): 79.15

Map:   0%|          | 0/1144 [00:00<?, ? examples/s]

0-99/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.96 | batch/sps: 100/2.84
100-199/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.94 | batch/sps: 100/3.04
200-299/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.97 | batch/sps: 100/2.89
300-399/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.96 | batch/sps: 100/4.80
400-499/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/7.00 | batch/sps: 100/0.63
500-599/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.97 | batch/sps: 100/3.90
600-699/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.99 | batch/sps: 100/3.31
700-799/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.98 | batch/sps: 100/2.34
800-899/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/7.06 | batch/sps: 100/3.88
900-999/1144 | total/used/cuda/res/ram (Gb): 79.15/73.25/1.52/71.90/6.99 | batch/sps: 100/3.10
1000-1099/1144 | total/used/cuda/res/ram (Gb): 79.15/

### Eval decoder-only models

In [7]:
%%time
%load_ext autoreload

batch_size = 20
max_length = 350


def model_process(batch, idx, **kwargs):
    num_samples = len(batch["input"])
    start_time = time.time()

    model = kwargs.get("model")
    tokenizer = kwargs.get("tokenizer")
    total_samples = kwargs.get("total_samples")

    # print(batch["input"])
    inputs = tokenizer(batch["input"], padding=True, return_tensors="pt").to(device)
    # print(inputs)
    # inputs = tokenizer(batch["input"], return_tensors="pt").inputs.to(device)
    # inputs = tokenizer(item["input"], return_tensors="pt").inputs

    # outputs = model.generate(inputs, max_length=512)
    outputs = model.generate(
        **inputs,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )
    # print(outputs)

    trimmed_output = outputs[:, inputs.input_ids.shape[1] :]
    processed = tokenizer.batch_decode(trimmed_output, skip_special_tokens=True)
    # print(processed)

    end_time = time.time()
    elapsed_time = end_time - start_time
    sps = num_samples / elapsed_time
    sps_str = f"{sps:.2f}"

    utilization = calculate_utilization()
    utilization_str = format_utilization_narrow(utilization)
    print(
        f"{idx[0]}-{idx[-1]}/{total_samples} | total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
        f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']} | "
        f"batch/sps: {num_samples}/{sps_str}"
    )

    # return {"processed": processed}
    return {"processed": processed}
    # return {"processed": processed, "utilization": utilization, "tps": tps}
    # return {"processed": processed, "utilization": utilization}


utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

processed_samples_map = {}

for task, samples in test_dataset_dict.items():
    total_samples = len(samples)

    print(f"Processing {total_samples} samples for {task}")

    start_time = time.time()

    processed_samples = samples.map(
        model_process,
        fn_kwargs={
            "model": model,
            "tokenizer": tokenizer,
            "total_samples": total_samples,
        },
        num_proc=1,
        batched=True,
        batch_size=batch_size,
        with_indices=True,
    )

    end_time = time.time()
    elapsed_time = end_time - start_time
    sps = total_samples / elapsed_time
    sps_str = f"{sps:.2f}"

    print(f"Finished processing {total_samples} samples for {task}.")
    print(processed_samples)
    print(processed_samples["reference"][0])
    print(processed_samples["processed"][0])

    utilization = calculate_utilization()
    utilization_str = format_utilization_narrow(utilization)
    print(
        f"{total_samples} | total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
        f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']} | "
        f"sps: {sps_str}"
    )

    processed_samples_map[task] = {
        # "task": task,
        # "samples": samples,
        "samples": processed_samples,
        "total_samples": total_samples,
        "elapsed_time": elapsed_time,
        "sps": sps,
        "batch_size": batch_size,
        "max_length": max_length,
        "utilization": utilization,
    }

# processed_samples = samples.map(model_process, num_proc=torch.cuda.device_count())
# processed_samples = samples.map(
#     model_process,
#     fn_kwargs={
#         "model": model,
#         "tokenizer": tokenizer,
#         "total_samples": total_samples,
#     },
#     num_proc=1,
#     batched=True,
#     batch_size=batch_size,
#     with_indices=True,
# )

processed_gec_samples = processed_samples_map["gec"]["samples"]

pprint(processed_gec_samples)
pprint(processed_gec_samples["processed"][:2])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
total/used/cuda/res/ram (Gb): 10.00/9.04/3.03/3.16/16.22
Processing 2 samples for simplification


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

0-1/2 | total/used/cuda/res/ram (Gb): 10.00/9.12/3.04/3.18/21.69 | batch/sps: 2/3.07
Finished processing 2 samples for simplification.
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 2
})
It is relatively safe to take linezolid for short periods.

2 | total/used/cuda/res/ram (Gb): 10.00/9.12/3.04/3.18/21.69 | sps: 0.33
Processing 2 samples for coherence


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

0-1/2 | total/used/cuda/res/ram (Gb): 10.00/9.19/3.04/3.24/22.03 | batch/sps: 2/12.34
Finished processing 2 samples for coherence.
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 2
})
Silver went to Japan's Miho Takagi in a time of 1: 54.55, while bronze went to Marrit Leenstra of the Netherlands in a time of 1.
54.55.
2 | total/used/cuda/res/ram (Gb): 10.00/9.19/3.04/3.24/21.63 | sps: 0.60
Processing 1 samples for clarity


...

0-1/2 | total/used/cuda/res/ram (Gb): 10.00/9.14/3.04/3.24/21.65 | batch/sps: 2/43.88
Finished processing 2 samples for paraphrase.
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 2
})
Yes, I've got a bike.

2 | total/used/cuda/res/ram (Gb): 10.00/9.14/3.04/3.24/21.65 | sps: 0.65
Processing 3 samples for gec


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

0-2/3 | total/used/cuda/res/ram (Gb): 10.00/9.31/3.04/3.45/21.92 | batch/sps: 3/11.10
Finished processing 3 samples for gec.
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 3
})
The romance is just a small part where Batman has to choose whether to be a hero or go with his girlfriend. I believe other people would also love this movie because it's not only thrilling, it makes you feel like you are in the movie, experiencing what is going on in the movie.

3 | total/used/cuda/res/ram (Gb): 10.00/9.31/3.04/3.45/21.92 | sps: 0.80
Processing 2 samples for neutralize


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

0-1/2 | total/used/cuda/res/ram (Gb): 10.00/9.32/3.04/3.45/21.66 | batch/sps: 2/37.15
Finished processing 2 samples for neutralize.
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 2
})
on 29 june 2007 price gave birth to her third child, a daughter named princess tiamii.

2 | total/used/cuda/res/ram (Gb): 10.00/9.32/3.04/3.45/21.66 | sps: 0.69
Dataset({
    features: ['task', 'input', 'reference', 'references', 'processed'],
    num_rows: 3
})
['', '']
CPU times: user 17 s, sys: 5.44 s, total: 22.4 s
Wall time: 22.4 s


### Saving eval results

In [11]:
hardware = "HomeDesktop (RTX3080)"
hardware = "NC24 (A100)"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

total_params = sum(p.numel() for p in model.parameters())
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total/trainable params: {total_params}/{total_trainable_params}")

all_flats = []
all_scores = []
if os.path.exists("results/all-scores.csv"):
    all_scores = pd.read_csv("results/all-scores.csv").to_dict("records")

all_fulls = []


rouge_metric = evaluate.load("rouge")
# glue_metric = evaluate.load("glue", "stsb")
sacreblue_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")
em_metric = evaluate.load("exact_match")


def calculate_scores(processed_samples):
    rouge_score = rouge_metric.compute(
        predictions=processed_samples["processed"], references=processed_samples["references"]
    )
    # pprint(rouge_score)

    sacreblue_score = sacreblue_metric.compute(
        predictions=processed_samples["processed"], references=processed_samples["references"]
    )
    # pprint(sacreblue_score)

    sari_score = sari_metric.compute(
        sources=processed_samples["input"],
        predictions=processed_samples["processed"],
        references=processed_samples["references"],
    )
    # pprint(sari_score)

    em_score = em_metric.compute(predictions=processed_samples["processed"], references=processed_samples["reference"])
    # pprint(em_score)

    return {
        "rouge": rouge_score,
        "sacreblue": sacreblue_score,
        "sari": sari_score,
        "em": em_score,
    }


for task, obj in processed_samples_map.items():
    print(f"task: {task}, samples: {len(obj['samples'])}")

    batch = obj["samples"]
    total_samples = len(batch)

    all_saved_samples = batch.remove_columns(["references"])
    saved_samples = all_saved_samples[:100] if len(all_saved_samples) > 100 else all_saved_samples
    flats_frame = pd.DataFrame.from_records(saved_samples)
    flats_frame.to_json(f"samples/{model_path}_{task}.json", orient="records")

    scores = calculate_scores(batch)
    # pprint(scores)

    score_paths = [
        "rouge.rouge1",
        # "rouge.rouge2",
        # "rouge.rougeL",
        # "rouge.rougeLsum",
        "sacreblue.score",
        "sari.sari",
        "em.exact_match",
    ]

    normalized_scores = {}
    for s, v in scores.items():
        print(f"s: {s}, v: {v}")
        for ss, vv in v.items():
            if not isinstance(vv, list):
                # normalized_scores[f"score.{k}.{ss}"] = vv
                path = f"{s}.{ss}"
                if path in score_paths:
                    normalized_scores[f"score.{s}.{ss}"] = vv
    # pprint(normalized_scores)

    normalized_utilization = {}
    for s, v in obj["utilization"].items():
        if not isinstance(v, list):
            normalized_utilization[f"utilization.{s}"] = v
    # print(normalized_utilization)

    flat_dict = {
        "model": model_name,
        "hardware": hardware,
        "total_params": total_params,
        "total_samples": total_samples,
        "elapsed_time": obj["elapsed_time"],
        "sps": obj["sps"],
        "batch_size": obj["batch_size"],
        "max_length": obj["max_length"],
        "task": task,
    }
    flat_dict.update(normalized_scores)
    flat_dict.update(normalized_utilization)
    # pprint(frame)

    all_flats.append(flat_dict)
    all_scores.append(flat_dict)

    fulls_frame = {
        "task": task,
        "total_samples": total_samples,
        "elapsed_time": obj["elapsed_time"],
        "sps": obj["sps"],
        "batch_size": obj["batch_size"],
        "max_length": obj["max_length"],
    }
    fulls_frame.update(
        {
            "scores": scores,
            "utilization": obj["utilization"],
        }
    )
    all_fulls.append(fulls_frame)

flats_frame = pd.DataFrame.from_records(all_flats)
flats_frame.to_csv(f"results/{model_path}.csv", index=False)

scores_frame = pd.DataFrame.from_records(all_scores)
scores_frame.to_csv(f"results/all-scores.csv", index=False)

fulls_dict = {
    "model": model_name,
    "hardware": hardware,
    "total_params": total_params,
}
for full in all_fulls:
    fulls_dict[full["task"]] = full

print(fulls_dict)
fulls_frame = pd.DataFrame.from_records([fulls_dict])
fulls_frame.to_json(f"results/{model_path}.json", orient="records", index=False)

model_name: bart-large-coedit,model_id: iliazlobin/bart-large-coedit,model_path: iliazlobin_bart-large-coedit
Total/trainable params: 406291456/406291456
task: paraphrase, samples: 1590
s: rouge, v: {'rouge1': 0.5116531557523737, 'rouge2': 0.23188210548668267, 'rougeL': 0.3444564830675132, 'rougeLsum': 0.3443783098073133}
s: sacreblue, v: {'score': 14.337135040972244, 'counts': [20761, 8188, 3713, 1715], 'totals': [42432, 40842, 39252, 37662], 'precisions': [48.927696078431374, 20.047989814406737, 9.459390604300419, 4.553661515586001], 'bp': 1.0, 'sys_len': 42432, 'ref_len': 41212}
s: sari, v: {'sari': 43.10653792855044}
s: em, v: {'exact_match': 0.0}
task: neutralize, samples: 1127
s: rouge, v: {'rouge1': 0.9332211826533825, 'rouge2': 0.8613339372215826, 'rougeL': 0.9314432064066901, 'rougeLsum': 0.9315545106748937}
s: sacreblue, v: {'score': 88.34870280800524, 'counts': [24837, 22609, 20551, 18658], 'totals': [26104, 24977, 23850, 22727], 'precisions': [95.146337726019, 90.5192777355